In [20]:
import pandas as pd
import mlxtend as ml
import numpy as np
import sklearn as sk
import sqlite3
import re
import math

In [21]:
ASSETS_DIR = "./assets"

WAREHOUSE_FILE = f"{ASSETS_DIR}/data_warehouse_2.csv"

pd.set_option("display.max_columns", None)  # Mostrar todas as colunas
pd.set_option("display.width", None)        # Ajustar a largura do display para não truncar

In [22]:
main_df = pd.read_csv(WAREHOUSE_FILE, index_col=False)

In [23]:
# main_df[["municipios", "area_coberta"]]
total_area = main_df["area_coberta"].sum()
main_df["area_coberta %"] = main_df["area_coberta"] / total_area

year_employenment_growth = main_df["empregados_2020"]     - main_df["empregados_2010"]
year_population_growth   = main_df["populacao_2020"]      - main_df["populacao_2010"]
year_gdp_growth          = main_df["pib_municipio_2020"]  - main_df["pib_municipio_2010"]
year_salary_delta        = main_df["massa_salarial_2020"] - main_df["massa_salarial_2010"]

main_df["delta_empregados"]= year_employenment_growth
main_df["delta_populacao"] = year_population_growth
main_df["delta_pib"]       = year_gdp_growth
main_df["delta_massa_salarial"]   = year_salary_delta

# main_df.sort_values(["taxa_de_populacao (2010-2020)", "taxa_de_empregados (2010-2020)", "area_coberta"], ascending=False)

In [28]:
total = len(main_df)

main_df = main_df.sort_values(by='area_coberta %', ascending=False)

def classification_thresholds(col) -> dict:
    classes = (
        ('low', 0.20),
        ('moderate', 0.50),
        ('good', 0.75),
        ('high', 0.95),
    )

    out = {
        'none': 0,
    }

    for key, val in classes:
        out[key] = col.quantile(val)

    return out

def classify_by_quantile(df, columns: list[str]):
    for col in columns:
        new_col = f'{col}_class'
        df[new_col] = 'none'
        classification = classification_thresholds(df[col])

        for i, row in df.iterrows():
            for label, threshold in classification.items():
                if row[col] >= threshold:
                    df.at[i, new_col] = label
    return df


filtered_df = main_df[main_df['area_coberta'] > 0].sort_values('area_coberta', ascending=False).copy()

filtered_df = classify_by_quantile(filtered_df, ['area_coberta', 'delta_populacao', 'delta_empregados', 'delta_pib', 'delta_massa_salarial'])



filtered_df

,municipios,quantidade_ucs,area_coberta,populacao_2010,populacao_2011,populacao_2012,populacao_2013,populacao_2014,populacao_2015,populacao_2016,populacao_2017,populacao_2018,populacao_2019,populacao_2020,massa_salarial_2010,massa_salarial_2011,massa_salarial_2012,massa_salarial_2013,massa_salarial_2014,massa_salarial_2015,massa_salarial_2016,massa_salarial_2017,massa_salarial_2018,massa_salarial_2019,massa_salarial_2020,pib_municipio_2010,pib_municipio_2011,pib_municipio_2012,pib_municipio_2013,pib_municipio_2014,pib_municipio_2015,pib_municipio_2016,pib_municipio_2017,pib_municipio_2018,pib_municipio_2019,pib_municipio_2020,empregados_2010,empregados_2011,empregados_2012,empregados_2013,empregados_2014,empregados_2015,empregados_2016,empregados_2017,empregados_2018,empregados_2019,empregados_2020,estabelecimentos,area_coberta %,delta_empregados,delta_populacao,delta_pib,delta_massa_salarial,area_coberta_class,delta_populacao_class,delta_empregados_class,delta_pib_class,delta_massa_salarial_class
88,FLORIANÓPOLIS,20,279607,427298,427298,433158,453285,461524,469690,477798,485838,492977,500973,508826,1.606721e+08,1.921229e+08,2.230646e+08,2.524728e+08,2.866640e+08,3.025580e+08,3.193570e+08,3.328831e+08,3.641840e+08,3.740900e+08,3.570488e+08,11276680000,12731618000,13946621000,14974993000,16915926000,17619984000,18660876000,19516694000,21059561000,21963928000,21312447000,127467,136037,141972,147047,153304,149780,146759,142896,147455,154647,147935,148,9.100383e-02,20468,81528,10035767000,1.963767e+08,high,high,high,high,high
189,PAULO LOPES,6,244407,6751,6751,6808,7045,7124,7203,7282,7360,7418,7494,7569,1.375521e+06,1.523021e+06,1.616975e+06,1.979181e+06,2.275203e+06,2.328851e+06,2.430322e+06,2.566049e+06,2.532010e+06,2.628638e+06,2.685109e+06,130616200,140022600,158650100,175388300,196747700,166876600,173173000,181154500,190304200,200456000,227628000,1212,1225,1215,1367,1440,1403,1323,1317,1288,1265,1292,721,7.954727e-02,80,818,97011800,1.309588e+06,high,low,low,low,low
181,PALHOÇA,4,244108,139990,139990,142558,150623,154244,157833,161395,164926,168259,171797,175272,2.744984e+07,3.396230e+07,4.032309e+07,4.648295e+07,5.446085e+07,5.869772e+07,6.263909e+07,6.739491e+07,7.626485e+07,7.863092e+07,8.277773e+07,2685689000,3133473000,3846978000,3990380000,4694889000,5104686000,4375512000,4834031000,5489676000,5985390000,6530836000,26622,28726,30927,32041,33701,32763,31762,32329,35785,37545,40187,1954,7.944995e-02,13565,35282,3845147000,5.532789e+07,high,good,high,good,good
94,GAROPABA,2,160199,18520,18520,18890,20024,20545,21061,21573,22082,22568,23078,23579,3.040572e+06,3.609895e+06,4.542455e+06,5.489691e+06,6.180405e+06,6.713530e+06,7.419670e+06,8.191510e+06,8.875684e+06,9.274646e+06,9.343595e+06,231533900,261002600,303007000,346584200,405903100,439794700,478206100,534236700,584882600,617279000,684966000,3068,3261,3482,3685,3892,4021,4234,4526,4818,4952,5081,1532,5.214005e-02,2013,5059,453432100,6.303022e+06,high,moderate,moderate,moderate,moderate
133,JAGUARUNA,1,154859,17496,17496,17695,18425,18704,18980,19254,19527,19755,20024,20288,1.953198e+06,2.278768e+06,2.666338e+06,3.188368e+06,3.681468e+06,4.499418e+06,4.937446e+06,5.239406e+06,6.108870e+06,6.408484e+06,6.765052e+06,221481600,239017600,259030600,304926600,360336800,401205200,456297800,473335500,469914200,581543000,813588000,2269,2454,2541,2750,2897,3081,3129,3116,3275,3527,3884,101692,5.040204e-02,1615,2792,592106400,4.811855e+06,high,moderate,moderate,moderate,moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,MAFRA,1,5,53141,53141,53361,54708,55012,55313,55611,55907,56017,56292,56561,8.915047e+06,9.891913e+06,1.162229e+07,1.364351e+07,1.490032e+07,1.580058e+07,1.703784e+07,2.069527e+07,2.145804e+07,2.130827e+07,2.155910e+07,1001513000,1056708000,1194126000,1254306000,144466100

In [27]:
keep = [
    'municipios', 'quantidade_ucs', 'estabelecimentos',
    'area_coberta',       'delta_pib',       'delta_populacao',       'delta_massa_salarial',
    'area_coberta_class', 'delta_pib_class', 'delta_populacao_class', 'delta_massa_salarial_class',
]

out_df = pd.DataFrame()
for col in keep:
    out_df[col] = filtered_df[col].copy()

out_df.to_csv('data_warehouse_2_discretized.csv')